In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
from AutoCordon.cordon import get_road_closure_locations, split_roads_with_cordon, remove_roads_within_cordon, extract_closure_candidates
from AutoCordon.buffer_zone import BufferZone
from pygeos.geometry import get_x, get_y
from pygeos.io import from_shapely
from pygeos.predicates import equals
import momepy
import networkx as nx

In [ ]:
roads = gpd.read_file(r"..\tests\data\sample_roads_soton_centre.geojson").explode().reset_index(drop=True)
road_lines = from_shapely(roads.geometry)

In [ ]:
remaining_roads
distance = 550
distance_max = 750
closure_locations = get_road_closure_locations(centre, distance, road_lines)
split_roads = split_roads_with_cordon(centre, distance, road_lines)
new_lines = [road for road in split_roads if not equals(road, road_lines).any()]
new_lines_gdf = gpd.GeoDataFrame({"geometry": new_lines, "id": range(len(new_lines))})
# remaining_roads = extract_closure_candidates(centre, distance, distance_max, road_lines)
# remaining_roads = remove_roads_within_cordon(centre, distance, road_lines)
remaining_roads_gdf = gpd.GeoDataFrame({"geometry": remaining_roads, "id": range(len(remaining_roads))}).explode().reset_index()

In [ ]:
remaining_roads_gdf

In [ ]:
fig, ax = plt.subplots(figsize=[15, 15])
roads.plot(ax=ax, color="grey")
remaining_roads_gdf.plot(color="r", ax=ax)
ax.scatter(get_x(closure_locations), get_y(closure_locations), color='y')
ax.scatter(*centre, color='g')
plt.show()

In [ ]:
bz = BufferZone(centre, distance, distance_max)
remaining_roads = bz.get_intersecting_lines(road_lines)
interior_closure_points = bz.get_intersecting_perimeter_points("interior", road_lines)
exterior_closure_points = bz.get_intersecting_perimeter_points("exterior", road_lines)
remaining_roads_gdf = gpd.GeoDataFrame({"geometry": remaining_roads, "id": range(len(remaining_roads))})

fig, ax = plt.subplots(figsize=[15, 15])
roads.plot(ax=ax, color="grey")
remaining_roads_gdf.plot(color="r", ax=ax)
ax.scatter(get_x(interior_closure_points), get_y(interior_closure_points), color='blueviolet')
ax.scatter(get_x(exterior_closure_points), get_y(exterior_closure_points), color='c')
ax.scatter(*centre, color='lime')
plt.show()

In [ ]:
# split_roads_gdf = gpd.GeoDataFrame({"geometry": split_roads, "id": range(len(split_roads))})
graph = momepy.gdf_to_nx(remaining_roads_gdf.explode().reset_index(drop=True))

In [ ]:
nodes, edges, sw = momepy.nx_to_gdf(graph, points=True, lines=True,
                                    spatial_weights=True)
nodes
sw

In [ ]:
import networkx as nx 

In [ ]:
graph.nodes

In [ ]:
source = (442745.0, 111065.0)
target = (441391.415, 111891.2990000006)
paths = [p for p in nx.all_shortest_paths(graph, source=source, target=target)]
x, y = zip(*paths[0])

In [ ]:
fig, ax = plt.subplots(figsize=[15, 15])
# roads.plot(ax=ax)
remaining_roads_gdf.plot(ax=ax)
# new_lines_gdf.plot(color="r", ax=ax)
ax.scatter(x, y, color='r')
ax.scatter(*centre, color='g')
ax.scatter(*source, color='g')
ax.scatter(*target, color='g')
plt.show()

In [ ]:
import networkx as nx
import momepy

load_centrality = nx.load_centrality(graph)

In [ ]:
fig, ax = plt.subplots(figsize=[15, 15])
load_centrality_gdf = gpd.GeoDataFrame({"geometry": gpd.points_from_xy(*zip(*load_centrality)),
                                        "load_centrality": load_centrality.values()})
remaining_roads_gdf.plot(color="grey", ax=ax)
load_centrality_gdf.plot("load_centrality", ax=ax)

In [ ]:
components = nx.connected_components(graph)
# list(nx.connected_components(graph.subgraph(component)))
node_list = []
nc_list = []
for component in components:
    centrality = nx.degree_centrality(nx.Graph(graph.subgraph(component).edges()))
    min_c = min(centrality.values())
    max_c = max(centrality.values())
    if max_c == 0:
        continue
    nc_list.extend([c / max_c for c in centrality.values()])
#     nc_list.extend(centrality.values())
    node_list.extend(centrality.keys())
fig, ax = plt.subplots(figsize=[15, 15])
norm_centrality_gdf = gpd.GeoDataFrame({"geometry": gpd.points_from_xy(*zip(*node_list)),
                                        "centrality": nc_list})
remaining_roads_gdf.plot(color="grey", ax=ax)
norm_centrality_gdf.plot("centrality", ax=ax, legend=True)

In [ ]:
coords = []
component_id = []
for n, component in enumerate(nx.connected_components(graph)):
    coords.extend(gpd.points_from_xy(*zip(*component)))
    component_id.extend([n] * len(component)) 

fig, ax = plt.subplots(figsize=[15, 15])
remaining_roads_gdf.plot(color="grey", ax=ax)
components_gdf = gpd.GeoDataFrame({"geometry": coords, "component_id": component_id})
components_gdf.plot("component_id", ax=ax, cmap="tab20")
ax.scatter(get_x(interior_closure_points), get_y(interior_closure_points), color='r')
ax.scatter(get_x(exterior_closure_points), get_y(exterior_closure_points), color='g')


In [ ]:
components = list(nx.connected_components(graph))

component_id = 6
subgraph_gdf = components_gdf[components_gdf["component_id"] == component_id]
print(components[component_id])
axis_buffer = 50
fig, ax = plt.subplots(figsize=[15, 15])
remaining_roads_gdf.plot(color="grey", ax=ax)
subgraph_gdf.plot("component_id", ax=ax, color="c")
ax.scatter(get_x(interior_closure_points), get_y(interior_closure_points), color='r')
ax.scatter(get_x(exterior_closure_points), get_y(exterior_closure_points), color='r')
subgraph_bounds = subgraph_gdf.total_bounds
ax.set_xlim(subgraph_bounds[0] - axis_buffer, subgraph_bounds[2] + axis_buffer)
ax.set_ylim(subgraph_bounds[1] - axis_buffer, subgraph_bounds[3] + axis_buffer)
plt.show()

In [ ]:
from pygeos.coordinates import get_coordinates
from networkx.algorithms.connectivity import minimum_st_node_cut

interior_closure_coords = {(x, y) for x,y in get_coordinates(interior_closure_points)}
exterior_closure_coords = {(x, y) for x,y in get_coordinates(exterior_closure_points)}

interior_nodes = list(components[component_id].intersection(interior_closure_coords))
exterior_nodes = list(components[component_id].intersection(exterior_closure_coords))

min_cut = nx.node_disjoint_paths(graph.subgraph(components[component_id]), interior_nodes[0], exterior_nodes[1])
for n, cut in enumerate(min_cut):
    print(n)
    print(cut)

In [ ]:
fig, ax = plt.subplots(figsize=[15, 15])
remaining_roads_gdf.plot(color="grey", ax=ax)
subgraph_gdf.plot( ax=ax, color="lightsteelblue")
ax.scatter(get_x(interior_closure_points), get_y(interior_closure_points), color='orange')
ax.scatter(get_x(exterior_closure_points), get_y(exterior_closure_points), color='orange')
subgraph_bounds = subgraph_gdf.total_bounds
ax.set_xlim(subgraph_bounds[0] - axis_buffer, subgraph_bounds[2] + axis_buffer)
ax.set_ylim(subgraph_bounds[1] - axis_buffer, subgraph_bounds[3] + axis_buffer)
ax.scatter(*zip(*min_cut[0]), color='lime')
ax.scatter(*interior_nodes[0], color='r')
ax.scatter(*exterior_nodes[0], color='r')
plt.show()

In [ ]:
# from networkx.algorithms import approximation as apxa
components = list(nx.connected_components(graph))

component_id = 6
# k_comps = apxa.steiner_tree((nx.Graph(graph.subgraph(components[component_id]).edges())), list(interior_closure_coords) + list(exterior_closure_coords))
# print(k_comps)
# bridges = nx.bridges(nx.Graph(graph.subgraph(components[component_id])))
# nodes1, nodes2 = zip(*list(bridges))
# bridge_nodes = nodes1 + nodes2
# nx.draw(nx.Graph(nx.bridges(graph.subgraph(components[component_id]))))
subgraph = graph.subgraph(components[component_id])
# contracted_graph = nx.contracted_nodes(subgraph, interior_nodes[0], interior_nodes[1])
middle_nodes = set(subgraph.nodes()).difference(set(interior_nodes + exterior_nodes))
M = nx.all_pairs_node_connectivity(subgraph, interior_nodes + exterior_nodes)
print(M)
m_nodes = [list(node) for node in M.nodes]
fig, ax = plt.subplots(figsize=[15, 15])
ax.scatter(*zip(*subgraph.nodes()), color="red")
for line in subgraph.edges():
    ax.plot(*(zip(*line)), color="grey")
for m_node in m_nodes:
    print(*zip(*m_node))
    ax.scatter(*zip(*m_node))
for line in M.edges():
    ax.plot(*(zip(*line)), color="orange")

# nx.draw(M)

In [ ]:
fig, ax = plt.subplots(figsize=[15, 15])
remaining_roads_gdf.plot(color="grey", ax=ax)
subgraph_gdf.plot( ax=ax, color="lightsteelblue")
ax.scatter(get_x(interior_closure_points), get_y(interior_closure_points), color='orange')
ax.scatter(get_x(exterior_closure_points), get_y(exterior_closure_points), color='orange')
subgraph_bounds = subgraph_gdf.total_bounds
ax.set_xlim(subgraph_bounds[0] - axis_buffer, subgraph_bounds[2] + axis_buffer)
ax.set_ylim(subgraph_bounds[1] - axis_buffer, subgraph_bounds[3] + axis_buffer)
ax.scatter(*zip(*bridge_nodes), color='lime')
ax.scatter(*interior_nodes[0], color='r')
ax.scatter(*exterior_nodes[0], color='r')
plt.show()

In [ ]:
graph_degree = momepy.node_degree(subgraph)
graph_degree.nodes[(442436.0, 111548.0)]['degree']
degree_subgraph = momepy.nx_to_gdf(graph_degree)


In [ ]:
from pygeos.measurement import distance
from pygeos.creation import points
from pygeos.io import from_shapely

degree_subgraph[0]['dist_to_centre'] = distance(from_shapely(degree_subgraph[0].geometry), points(centre))
degree_subgraph

In [ ]:
# find the degree of all the interior closure nodes

# find their next neighbour, +1 for closures behind that node

# would be easier if was directed

In [ ]:
fig, ax = plt.subplots(figsize=[15, 15])
remaining_roads_gdf.plot(color="grey", ax=ax)
degree_subgraph[0].plot('degree', ax=ax)
subgraph_bounds = degree_subgraph[0].total_bounds
ax.set_xlim(subgraph_bounds[0] - axis_buffer, subgraph_bounds[2] + axis_buffer)
ax.set_ylim(subgraph_bounds[1] - axis_buffer, subgraph_bounds[3] + axis_buffer)

In [ ]:
a, b = zip(*list(subgraph.degree))
print(a, b)

In [ ]:
from AutoCordon.cordon_graph import make_buffer_zone_graph
from AutoCordon.buffer_zone import BufferZone
from pygeos.io import from_shapely
import geopandas as gpd

roads = gpd.read_file(r"..\tests\data\sample_roads_soton_centre.geojson").explode().reset_index(drop=True)
road_lines = from_shapely(roads.geometry)
distance = 550
distance_max = 750
centre = (442000, 112000)

bz = BufferZone(centre, distance, distance_max)
remaining_roads = bz.get_intersecting_lines(road_lines)
interior_closure_points = bz.get_intersecting_perimeter_points("interior", road_lines)
exterior_closure_points = bz.get_intersecting_perimeter_points("exterior", road_lines)

remaining_roads_gdf = gpd.GeoDataFrame({"geometry": remaining_roads, "id": range(len(remaining_roads))}).explode().reset_index()

g = make_buffer_zone_graph(centre, remaining_roads)

In [ ]:
g.nodes(data="interior_closure", default=False)
g.nodes(data="exterior_closure", default=False)
for point in interior_closure_points:
    print(g.nodes[point])
    g.nodes[point]["interior_closure"] = True
for point in exterior_closure_points:
    g.nodes[point]["exterior_closure"] = True


In [ ]:
import networkx as nx
node_attrs = nx.get_node_attributes(g, "dist_to_centre")
node_gdf = gpd.GeoDataFrame({"geometry": node_attrs.keys(),
                 "dist_to_centre":  node_attrs.values()})
inital_closure_nodes = nx.get_node_attributes(g, "initial_closure")
inital_closure_gdf = gpd.GeoDataFrame({"geometry": inital_closure_nodes.keys(),
                 "initial_closure":  inital_closure_nodes.values()})
max_closure_nodes = nx.get_node_attributes(g, "exterior_closure")
max_closure_nodes_gdf = gpd.GeoDataFrame({"geometry": max_closure_nodes.keys(),
                 "initial_closure":  max_closure_nodes.values()})
node_degree = dict(g.degree())
degree_gdf = gpd.GeoDataFrame({"geometry": node_degree.keys(),
                 "degree":  node_degree.values()})

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=[15, 15])
remaining_roads_gdf.plot(color="grey", ax=ax)
node_gdf.plot("dist_to_centre", ax=ax)
inital_closure_gdf.plot("dist_to_centre", ax=ax, color='r')

In [ ]:
fig, ax = plt.subplots(figsize=[15, 15])
remaining_roads_gdf.plot(color="grey", ax=ax)
degree_gdf.plot("degree", ax=ax, color="white")
degree_gdf[degree_gdf['degree'] > 1].plot("degree", ax=ax)
# inital_closure_gdf.plot("dist_to_centre", ax=ax, color='r')

In [ ]:
# remove subgraphs that don't go all the way through... I think
coords = []
component_id = []
for n, component in enumerate(nx.connected_components(nx.to_undirected(g))):
#     print(g.nodes[list(component)[0]])
    for node in component:
        g.nodes[node]["subgraph"] = n
        if node in inital_closure_nodes:
            if node in max_closure_nodes:
                coords.extend(component)
                compnent_id.extend([n] * len(compnent))
                break
                
    print(g.nodes(data=True))

In [ ]:
subgraphs = {}
for n, component in enumerate(nx.connected_components(nx.to_undirected(g))):
    subgraphs[n] = list(component)
    for node in component:
        g.nodes[node]["subgraph"] = n

# print(subgraphs)
invalid_subgraphs = []
for subgraph_id, nodes in subgraphs.items():
    contains_interior = set(interior_closure_points).intersection(set(nodes))
    contains_exterior = set(exterior_closure_points).intersection(set(nodes))
    if not contains_interior or not contains_exterior:
        invalid_subgraphs.append(subgraph_id)
        
invalid_subgraphs
set(g)

In [ ]:
interior_set = set(interior_closure_points)
exterior_set = set(exterior_closure_points)
subgraphs = {}
g.nodes(data="subgraph", default=None)
for subgraph_id, component in enumerate(nx.connected_components(nx.to_undirected(g))):
    contains_interior = interior_set.intersection(component)
    contains_exterior = exterior_set.intersection(component)
    if contains_interior and contains_exterior:
        subgraph_nodes = list(component)
        subgraphs[subgraph_id] = subgraph_nodes
        for node in subgraph_nodes:
            g.nodes[node]["subgraph"] = subgraph_id
g.subgraph(subgraphs[1]).nodes(data=True)

In [ ]:
g.nodes(data="interior_closure", default=False)
g.nodes(data="exterior_closure", default=False)
interior_set = set(interior_closure_points)
exterior_set = set(exterior_closure_points)
subgraphs = {}
components = nx.connected_components(nx.to_undirected(g))
for subgraph_id, component in enumerate(components):
    interior_nodes = interior_set.intersection(component)
    exterior_nodes = exterior_set.intersection(component)

    if interior_nodes and exterior_nodes:
        subgraph = g.subgraph(list(component)).copy()
        for node in interior_nodes:
            subgraph.nodes[node]["interior_closure"] = True
        for node in exterior_nodes:
            subgraph.nodes[node]["exterior_closure"] = True
        subgraph.graph["subgraph_id"] = subgraph_id
        subgraphs[subgraph_id] = subgraph
subgraphs

In [ ]:
from AutoCordon.cordon_graph import get_valid_subgraphs
get_valid_subgraphs(g, interior_closure_points, exterior_closure_points)